# Library Install

In [1]:
!pip install pydantic-ai > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.43.0 which is incompatible.
google-auth-oauthlib 1.2.3 requires google-auth<2.42.0,>=2.15.0, but you have google-auth 2.43.0 which is incompatible.


In [2]:
from google.colab import userdata
from pydantic_ai import Agent
import os

# API parameters

In [3]:
os.environ["GEMINI_MODEL"]  = "gemini-2.5-flash-lite"
os.environ["GEMINI_KEY"] = userdata.get('GeminiKey')

# os.environ["OR_MODEL"]  ="minimax/minimax-m2"
# os.environ["OR_KEY"] = userdata.get('ORKey')

# os.environ["CEREBRAS_MODEL"]  = "gpt-oss-120b"
# os.environ["CEREBRAS_KEY"] = userdata.get('CereKey')

# os.environ["GROQ_MODEL"]  = "llama-3.3-70b-versatile"
# os.environ["GROQ_KEY"] = userdata.get('GroqKey')

In [4]:
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider
provider = GoogleProvider(api_key=os.environ["GEMINI_KEY"])
model = GoogleModel(os.environ["GEMINI_MODEL"], provider=provider)

In [5]:
from IPython.display import display, Markdown

def nbprint(s):
    display(Markdown(s))

# Zero-Shot Prompting - The “I Trust You” Approach

In [17]:
zero_shot_agent = Agent(
    model=model,
    system_prompt="You are an addition calculator. Output only the final number."
)
response = await zero_shot_agent.run("45 + 77")
nbprint(response.output)

122

# Few-Shot Prompting - “Here, Copy My Homework”

In [18]:
few_shot_agent = Agent(
    model=model,
    system_prompt="""
You solve additions like this:
Q: 10 + 5 → A: 15
Q: 7 + 8 → A: 15
Now follow the same format."""
)
response = await few_shot_agent.run("45 + 77")
nbprint(response.output)

A: 122

# Chain-of-Thought - “Explain Like I'm Slow” Mode

In [19]:
cot_agent = Agent(
    model=model,
    system_prompt="Explain your reasoning step-by-step, then give the final answer."
)
response = await cot_agent.run("Add 45 and 77.")
nbprint(response.output)

1. **Identify the operation:** The problem asks to "add", which means we need to perform addition.
2. **Identify the numbers:** The numbers to be added are 45 and 77.
3. **Set up the addition:** We can set up the addition vertically, aligning the numbers by place value (ones place under ones place, tens place under tens place).

   ```
     45
   + 77
   ----
   ```

4. **Add the ones column:** Add the digits in the ones place: 5 + 7 = 12.
   * Write down the '2' in the ones place of the answer.
   * "Carry over" the '1' to the tens column.

   ```
     ¹45
   + 77
   ----
      2
   ```

5. **Add the tens column:** Add the digits in the tens place, including the carried-over '1': 1 + 4 + 7 = 12.
   * Write down the '2' in the tens place of the answer.
   * Since there are no more columns, write down the '1' in the hundreds place of the answer.

   ```
     ¹45
   + 77
   ----
    122
   ```

6. **State the final answer:** The sum of 45 and 77 is 122.

The final answer is $\boxed{122}$.

# Role Prompting - “Become Someone Else (Legally)”

In [20]:
role_agent = Agent(
    model=model,
    system_prompt="Act like a friendly batman. Explain things in short poem."
)
response = await role_agent.run("How do you add 45 and 77?")
nbprint(response.output)


Fear not, citizen, for I shall show the way,
To add these numbers and brighten your day!

Forty-five, a number bold and bright,
Seventy-seven, shining with all its might.

Let's line them up, with care and a plan,
Units below units, like a helpful hand.

5 and 7, together they do meet,
A total of 12, oh, what a treat!
The '2' stays put, a simple display,
The '1' ascends, to join the next fray.

Now, the tens, a sturdy brigade,
4 and 7, a sum that's made.
With the '1' from before, a helpful guest,
8, 9, 10, 11, 12, putting hearts to the test!

So, the answer rings out, clear and true,
One hundred and twenty-two, for me and for you!

# Contextual Prompting - “Here’s the Cheat Sheet”

In [21]:
context_agent = Agent(
    model=model,
    system_prompt="""
Use this rule:
Align digits → Add units → Add tens → Combine result.
Follow this rule strictly.
"""
)
response = await context_agent.run("45 + 77")
nbprint(response.output)

Align digits:
  45
+ 77

Add units: 5 + 7 = 12

Add tens: 4 + 7 = 11

Combine result:
The '12' from the units addition means 2 in the units place and a carry-over of 1 to the tens place.
The '11' from the tens addition, plus the carry-over of 1 from the units, makes 12 in the tens place.
So, 12 tens and 2 units.

This gives us 122.